## Data Cleaning

Before any regression or modeling can be done, we first need to create the data table by merging data from different sources, performing some cleanup, and using some natural language sentiment analysis on text information.

In [1]:
import numpy as np
import pandas as pd

First. import the Yelp business dataset from saved file. As you can see, the data has a unique business identifier, and some information on the business laid out in columns.

In [4]:
business_json_path = 'yelp_academic_dataset_business.json'

df_b = pd.read_json(business_json_path, lines = True)
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


Now, we import the scraped zipcode data to add onto each business.

In [5]:
scraped_data = pd.read_csv('Scraped_Data_V1.csv')
scraped_data.head()

,Unnamed: 0,Zip Code,Population,Median Household Income,Avg Commute Time,Unemployment Rate
0,0,80302,26941,$45733,18.9 Minutes,5.5%
1,0,97218,14561,$50070,24.4 Minutes,5.5%
2,0,97214,23813,$48887,21.7 Minutes,5.5%
3,0,32763,21263,$38372,26 Minutes,6.6%
4,0,30316,31110,$43026,28.2 Minutes,5.9%


We need to clean the data a bit, so we need to change the data type on some variables, as well remove any values that have missing information for median household income.

In [10]:
def clean_zipdata(scraped_data):
    scraped_data['Zip Code'] = scraped_data['Zip Code'].astype('str')

    def add_0(x):
        if len(x) == 4:
            return '0' + x
        else:
            return x
    scraped_data['Zip Code'] = scraped_data['Zip Code'].apply(add_0)

    cleaned_data = scraped_data.loc[scraped_data['Population'] != 0]
    cleaned_data['Median Household Income'] = cleaned_data['Median Household Income'].str.replace('$','')
    def make_0(x):
        if x == '':
            return 0
        else:
            return x
        
    cleaned_data['Median Household Income'] = cleaned_data['Median Household Income'].apply(make_0)
    cleaned_data['Median Household Income'] = cleaned_data['Median Household Income'].astype('int')
    
    cleaned_data['Avg Commute Time'] = cleaned_data['Avg Commute Time'].str.replace('Minutes','')
    cleaned_data['Avg Commute Time'] = cleaned_data['Avg Commute Time'].astype('float')
    
    cleaned_data['Median Household Income'] = cleaned_data['Median Household Income'].apply(make_0)
    cleaned_data['Unemployment Rate'] = cleaned_data['Unemployment Rate'].str.replace('%','').astype('float')

    return cleaned_data


In [11]:
cleaned_data = clean_zipdata(scraped_data)
cleaned_data.head()

/var/folders/n6/6skhpz0j0fjcd1gwv1lcydxc0000gp/T/ipykernel_50870/3130451140.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cleaned_data['Median Household Income'] = cleaned_data['Median Household Income'].str.replace('$','')
/Users/kenhua/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Unnamed: 0,Zip Code,Population,Median Household Income,Avg Commute Time,Unemployment Rate
0,0,80302,26941,45733,18.9,5.5
1,0,97218,14561,50070,24.4,5.5
2,0,97214,23813,48887,21.7,5.5
3,0,32763,21263,38372,26.0,6.6
4,0,30316,31110,43026,28.2,5.9


We now merge the yelp business dataframe with the cleaned-up zip dataframe

In [12]:
yelp_zip = df_b.merge(cleaned_data, left_on = 'postal_code', right_on = 'Zip Code', how = 'inner')

yelp_zip.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Unnamed: 0,Zip Code,Population,Median Household Income,Avg Commute Time,Unemployment Rate
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",0,80302,26941,45733,18.9,5.5
1,rYs_1pNB_RMtn5WQh55QDA,Chautauqua General Store,100 Clematis Dr,Boulder,CO,80302,39.998449,-105.281006,3.5,5,1,"{'BikeParking': 'True', 'RestaurantsTakeOut': ...","Food, Shopping, Convenience Stores, Souvenir S...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ...",0,80302,26941,45733,18.9,5.5
2,hfoHYhD4uzqsHQDxKMFnQw,Pickled Lemon,1155 13th St,Boulder,CO,80302,40.008353,-105.276600,4.5,24,0,"{'BusinessAcceptsCreditCards': 'True', 'Alcoho...","Middle Eastern, Restaurants","{'Monday': '11:0-15:0', 'Tuesday': '11:0-15:0'...",0,80302,26941,45733,18.9,5.5
3,S0j-zDUb1uSB6SPPVB8olw,Urban Studio Hair Design,2422 Arapahoe Ave,Boulder,CO,80302,40.014568,-105.262677,2.0,20,0,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Hair Salons, Shopping, Barbers, Cosmetics & Be...","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",0,80302,26941,45733,18.9,5.5
4,u1at3Wrovnm0628ziAGGDw,Zeal Optics,1230 Spruce St,Boulder,CO,80302,40.018850,-105.279758,4.5,10,1,"{'BusinessParking': '{'garage': True, 'street'...","Outdoor Gear, Eyewear & Opticians, Fashion, Sp...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",0,80302,26941,45733,18.9,5.5


Since the dataset currently is far too large, let's choose the subsection of the dataset based on Median Household Income Range. In this function:

Input: yelp_zip: Dataframe, income_class: 'high' or 'low', depending on if you want a high or low income, threshold: a threshold parameter to determine the offset from the mean.

Output: Filtered DataFrame based on above criteria

In [13]:
def choose_dataset(yelp_zip, income_class, threshold = 25000):
    if income_class == 'high':
        mean_income = yelp_zip['Median Household Income'].mean() + threshold
        high_income = yelp_zip.loc[yelp_zip['Median Household Income'] > mean_income]
        high_income_restaurants = high_income.loc[high_income['categories'].str.contains('Restaurants') == True]
        return high_income_restaurants
    elif income_class == 'low':
        mean_income = yelp_zip['Median Household Income'].mean() - threshold
        low_income = yelp_zip.loc[yelp_zip['Median Household Income'] < mean_income]
        low_income = low_income.loc[low_income['Median Household Income'] > 0]

        low_income_restaurants = low_income.loc[low_income['categories'].str.contains('Restaurants') == True]
        return low_income_restaurants


Let's choose low_income range

In [16]:
low_income = choose_dataset(yelp_zip, income_class = 'low', threshold = 25000)
low_income

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Unnamed: 0,Zip Code,Population,Median Household Income,Avg Commute Time,Unemployment Rate
2917,XWp7dwUDSlYKkuLcK77UsQ,Five Guys,898 Saxon Blvd,Orange City,FL,32763,28.910718,-81.289820,4.0,27,1,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Fast Food, Burgers, Restaurants, American (New)","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",0,32763,21263,38372,26.0,6.6
2922,w8HlVUfbMPPQt_9w6IPEgg,Jimmy John's,2631 Enterprise Rd,Orange City,FL,32763,28.914181,-81.291152,2.5,18,0,"{'BikeParking': 'True', 'GoodForKids': 'True',...","Delis, Restaurants, Sandwiches, Fast Food","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",0,32763,21263,38372,26.0,6.6
2923,mFlNpzZjqs13wytnTLOQOA,New York Buffet,2404 S Volusia Ave,Orange City,FL,32763,28.921736,-81.299298,1.0,5,0,"{'Alcohol': ''beer_and_wine'', 'RestaurantsRes...","Food, Buffets, Restaurants, American (Traditio...",None,0,32763,21263,38372,26.0,6.6
2929,ZGy36Ixt5x_3nww4PuDn9w,Pizza Hut,917 Saxon Blvd,Orange City,FL,32763,28.911728,-81.288593,2.5,16,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Pizza, Italian, Chicken Wings, Restaurants","{'Monday': '11:0-0:0', 'Tuesday': '11:0-0:0', ...",0,32763,21263,38372,26.0,6.6
2931,mfZJhnpeZsOSbJ09XQfcBQ,Sonny's BBQ,1024 Saxon Blvd.,Orange City,FL,32763,28.910649,-81.284014,3.0,41,1,"{'HasTV': 'True', 'Caters': 'True', 'Restauran...","Barbeque, Restaurants, Event Planning & Servic...","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3...",0,32763,21263,38372,26.0,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141536,-eswhv8XRPXfL7jsGBNDkQ,The Tavern,7251 Five Oaks Dr,Harmony,FL,34773,28.191197,-81.145025,3.5,29,1,"{'Caters': 'True', 'GoodForKids': 'True', 'Noi...","Restaurants, American (New), Comfort Food, Bur...","{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'...",0,34773,2983,37611,32.6,7.2
141537,tE-1zad2zUITk1T7_kAeAg,D'Oro Pizza,7174 Harmony Sq Dr S,Harmony,FL,34773,28.062985,-81.051463,4.0,10,1,"{'RestaurantsAttire': ''casual'', 'BikeParking...","Restaurants, Sandwiches, Pizza, Italian, Salad","{'Monday': '10:30-20:30', 'Tuesday': '10:30-20...",0,34773,2983,37611,32.6,7.2
141619,Mn7iwfUnmTcDOFm0z8eOLA,IHOP,165 Hargraves Dr,AUSTIN,TX,77837,30.206563,-97.975722,2.0,50,1,"{'BusinessAcceptsCreditCards': 'True', 'CoatCh...","Food, Breakfast & Brunch, American (Traditiona...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",0,77837,1587,23401,29.9,6.6
141698,Q7j-Ue40ZDX_CKPXjO_JBQ,Taqueria Los Chilangos,11817 Mancha Rd,Austin,TX,79748,30.147318,-97.832764,4.5,16,1,None,"Restaurants, Food, Mexican, Food Trucks","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",0,79748,173,32657,18.4,7.6


The next step to this part is to attach user comment data to each business. The user data is stored in a seperate json file and must be merged with our current existing dataframe. However, the file is very large and we do not have enough memory to load the entire file all at once. To overcome this we write a function that takes in the file_path, the business dataframe, and loads the review dataset in chunks. Then it matches the rows in each chunk with the business dataframe, and merges and saves them.

In [22]:
def full_review_dataset(review_json_path, business_df):
    
    size = 100000
    review = pd.read_json(review_json_path, lines=True,
                          dtype={'review_id':str,'user_id':str,
                                 'business_id':str,'stars':int,
                                 'date':str,'text':str,'useful':int,
                                 'funny':int,'cool':int},
                          chunksize=size)
    chunk_list = []
    for chunk_review in review:
        # Drop columns that aren't needed
        chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
        # Renaming column name to avoid conflict with business overall star rating
        chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
        # Inner merge with edited business file so only reviews related to the business remain
        chunk_merged = pd.merge(business_df, chunk_review, on='business_id', how='inner')
        # Show feedback on progress
        chunk_list.append(chunk_merged)
    # After trimming down the review file, concatenate all relevant data back to one dataframe
    df_merged = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
    return df_merged

In [21]:
low_merged = full_review_dataset('yelp_academic_dataset_review.json', low_income)
low_merged.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Unnamed: 0,Zip Code,Population,Median Household Income,Avg Commute Time,Unemployment Rate,user_id,review_stars,text,date
0,XWp7dwUDSlYKkuLcK77UsQ,Five Guys,898 Saxon Blvd,Orange City,FL,32763,28.910718,-81.289820,4.0,27,...,0,32763,21263,38372,26.0,6.6,OKF7bG72yeGfK-UDIdcH9A,3,So the Customer service was good. My burger th...,2017-05-01 02:34:26
1,w8HlVUfbMPPQt_9w6IPEgg,Jimmy John's,2631 Enterprise Rd,Orange City,FL,32763,28.914181,-81.291152,2.5,18,...,0,32763,21263,38372,26.0,6.6,RQBqXfU_2Yx_AgybwEOp_w,4,Looking for a quick tasty sandwich Jimmy Johns...,2012-09-10 15:41:51
2,w8HlVUfbMPPQt_9w6IPEgg,Jimmy John's,2631 Enterprise Rd,Orange City,FL,32763,28.914181,-81.291152,2.5,18,...,0,32763,21263,38372,26.0,6.6,DeER6nRvM_7HrzANt7rW_A,5,This place has always been great whether I eat...,2015-04-02 22:03:10
3,mFlNpzZjqs13wytnTLOQOA,New York Buffet,2404 S Volusia Ave,Orange City,FL,32763,28.921736,-81.299298,1.0,5,...,0,32763,21263,38372,26.0,6.6,BizlCP9owr5DA5izQJ8ZtQ,1,New York Buffet formally Stacy's Buffet / Stac...,2014-05-07 23:46:23
4,mFlNpzZjqs13wytnTLOQOA,New York Buffet,2404 S Volusia Ave,Orange City,FL,32763,28.921736,-81.299298,1.0,5,...,0,32763,21263,38372,26.0,6.6,bHJuwgGylDlMw1D9kF4oUw,1,"The worst buffet, you can imagine. Years back ...",2015-11-07 22:53:31
5,ZGy36Ixt5x_3nww4PuDn9w,Pizza Hut,917 Saxon Blvd,Orange City,FL,32763,28.911728,-81.288593,2.5,16,...,0,32763,21263,38372,26.0,6.6,dUrSyiqf_T1CcoXxEsuuNw,1,I have yet to receive my order so I cannot vou...,2015-03-04 23:11:47
6,ZGy36Ixt5x_3nww4PuDn9w,Pizza Hut,917 Saxon Blvd,Orange City,FL,32763,28.911728,-81.288593,2.5,16,...,0,32763,21263,38372,26.0,6.6,z0E8-omA1TK67-IL4zTowA,1,Are you kidding me ??? this drive-through look...,2016-01-18 19:16:10
7,PYH2RJzpOnwvqY1IYsIu8A,Arepas Grill,7545 N Lamar Blvd,Austin,TX,78752,30.343872,-97.714924,5.0,13,...,0,78752,18064,36697,24.1,4.5,KvlWq5aXUYas_M3c1MAMsQ,5,Finally got to try this place! Been driving by...,2018-02-25 00:48:10
8,HSpFgxTcf9lihq1R-pcytQ,Kim Phung Restaurant,7601 N Lamar Blvd,Austin,TX,78752,30.344123,-97.713921,3.5,318,...,0,78752,18064,36697,24.1,4.5,zlL_PbFTXjrGXhTbPsjmlQ,4,The $5 Wednesday night pho special (7pm to 9pm...,2013-09-04 00:20:45
9,HSpFgxTcf9lihq1R-pcytQ,Kim Phung Restaurant,7601 N Lamar Blvd,Austin,TX,78752,30.344123,-97.713921,3.5,318,...,0,78752,18064,36697,24.1,4.5,Bfm-WN7J7tf7vtiRJfNmIw,5,Always come here for my curry fix. Today had s...,2015-07-16 17:00:46


The final step that needs to be done is to add in columns that analyze the text.

In [26]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import re

def add_sentence_sentiment(df2, keyword_list, keyname):
    reg_keyword_list = []
    for keyword in keyword_list:
        first_letter = keyword[0]
        first_letter_capital = first_letter.upper()
        reg_string = '['  + keyword[0] + first_letter_capital + ']' + keyword[1:]
        reg_keyword_list.append(reg_string)

    analyser = SentimentIntensityAnalyzer()
    
    def sentiment_analyzer_scores(paragraph):
        sentence_list = nltk.tokenize.sent_tokenize(paragraph)
        scores = []
        for sentence in sentence_list:
            for keyword in reg_keyword_list:
                find_list = re.findall(keyword,sentence)
                if len(find_list) > 0:
                    score = analyser.polarity_scores(sentence)
                    score = score.get('compound')
                    scores.append(score)
                    break
        #score_df = pd.DataFrame(scores)                
        #print("{:-<40} {}".format(sentence, str(score)))
        
        if len(scores) > 0:
            return sum(scores)/len(scores)
        else:
            return np.nan
    
    df2[str(keyname)] = df2['text'].apply(sentiment_analyzer_scores)
    
    
    return df2


In [27]:
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['manager','management','operation','experience'], keyname = 'Management')
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['atmosphere','vibe','environment', 'ambience','mood'], keyname = 'Atmosphere')
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['clean','dirty','sanitary', 'unclean','spotless'], keyname = 'Sanitation')
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['fast','slow','quick', 'forever','quickly', 'so long', 'time'], keyname = 'Speed')
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['server','waiter','service', 'customer','rude', 'generous', 'kind', 'mean'], keyname = 'Service')
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['dish','quality','delicious', 'tasty','disgusting', 'edible', 'unedible', 'yum'], keyname = 'Food')
low_merged = add_sentence_sentiment(low_merged, keyword_list = ['delicacy','exquisite','fancy','pricey','high end', 'best', 'world class', 'top', 'expensive', 'ripoff', 'rip off', 'overprice'], keyname = 'High End')


In [28]:
low_merged.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,review_stars,text,date,Management,Atmosphere,Sanitation,Speed,Service,Food,High End
0,XWp7dwUDSlYKkuLcK77UsQ,Five Guys,898 Saxon Blvd,Orange City,FL,32763,28.910718,-81.289820,4.0,27,...,3,So the Customer service was good. My burger th...,2017-05-01 02:34:26,NaN,NaN,NaN,NaN,0.44040,NaN,NaN
1,w8HlVUfbMPPQt_9w6IPEgg,Jimmy John's,2631 Enterprise Rd,Orange City,FL,32763,28.914181,-81.291152,2.5,18,...,4,Looking for a quick tasty sandwich Jimmy Johns...,2012-09-10 15:41:51,NaN,NaN,NaN,0.00000,NaN,0.0,NaN
2,w8HlVUfbMPPQt_9w6IPEgg,Jimmy John's,2631 Enterprise Rd,Orange City,FL,32763,28.914181,-81.291152,2.5,18,...,5,This place has always been great whether I eat...,2015-04-02 22:03:10,0.7003,NaN,NaN,0.66105,0.66105,NaN,0.6682
3,mFlNpzZjqs13wytnTLOQOA,New York Buffet,2404 S Volusia Ave,Orange City,FL,32763,28.921736,-81.299298,1.0,5,...,1,New York Buffet formally Stacy's Buffet / Stac...,2014-05-07 23:46:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mFlNpzZjqs13wytnTLOQOA,New York Buffet,2404 S Volusia Ave,Orange City,FL,32763,28.921736,-81.299298,1.0,5,...,1,"The worst buffet, you can imagine. Years back ...",2015-11-07 22:53:31,0.0000,NaN,NaN,NaN,NaN,NaN,NaN


The final step is to group the data by business, and take the mean value of numerical columns. This averages out the information for each business, and gives us a dataset with rows that are specific to each business.

In [29]:
grouped_data = low_merged.groupby(['name']).mean()
grouped_data = grouped_data.loc[~grouped_data['review_stars'].isin([1,2,3,4,5])]

grouped_data = grouped_data[~grouped_data.isnull().any(axis=1)]

grouped_data.head()


,latitude,longitude,stars,review_count,is_open,Unnamed: 0,Population,Median Household Income,Avg Commute Time,Unemployment Rate,review_stars,Management,Atmosphere,Sanitation,Speed,Service,Food,High End
name,,,,,,,,,,,,,,,,,,
'Ohana,28.405163,-81.583242,4.0,1065.0,1.0,0.0,26.0,35000.0,0.0,6.1,4.041133,0.338374,0.557899,0.162988,0.234307,0.301519,0.458047,0.486640
081 Wood Fired Pizza,28.540765,-81.377648,4.5,20.0,0.0,0.0,12050.0,32992.0,19.7,6.2,4.700000,0.077200,0.624900,0.340000,0.540500,0.388450,0.407321,0.462420
10 Barrel Brewing Portland,45.525984,-122.685468,4.0,942.0,1.0,0.0,14950.0,30833.0,23.3,5.5,3.955115,0.240997,0.597809,0.506141,0.306973,0.344668,0.550435,0.418221
101 By Teahaus,30.337121,-97.716716,4.0,256.0,1.0,0.0,18064.0,36697.0,24.1,4.5,3.866920,0.246808,0.603185,0.625565,0.272090,0.399359,0.403279,0.365481
11th Street Station,30.269190,-97.729194,3.0,39.0,0.0,0.0,21334.0,27695.0,21.1,4.5,3.179487,0.205975,0.288275,0.149440,0.235328,0.063539,0.235375,0.442269
